In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('tmdb-movie/tmdb_5000_movies.csv')
df1 = pd.read_csv('tmdb-movie/tmdb_5000_credits.csv')

In [3]:
df1.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [4]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
df.rename(columns={'id':'movie_id'},inplace=True)

In [6]:
df = pd.merge(df,df1,on='movie_id')

In [7]:
df.head(2)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [8]:
df.columns

Index(['budget', 'genres', 'homepage', 'movie_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title_x', 'vote_average', 'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

In [9]:
df.drop(columns=['budget','homepage','original_language','revenue', 'runtime','spoken_languages','status','title_x','title_y'],inplace=True)

In [10]:
df.head(1)

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,release_date,tagline,vote_average,vote_count,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
def json_decode(data,key):
    result=[]
    data=json.loads(data)
    for item in data:
        result.append(item[key])
    return (result)

In [12]:
df['genres']=df['genres'].apply(json_decode,key='name')

In [13]:
df['keywords']=df['keywords'].apply(json_decode,key='name')

In [14]:
df['production_companies']=df['production_companies'].apply(json_decode,key='name')

In [15]:
df['production_countries']=df['production_countries'].apply(json_decode,key='name')

In [16]:
df['cast']=df['cast'].apply(json_decode,key='name')

In [17]:
df['crew']=df['crew'].apply(json_decode,key='name')

In [18]:
df.head(1)

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,release_date,tagline,vote_average,vote_count,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,Enter the World of Pandora.,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 14 columns):
genres                  4803 non-null object
movie_id                4803 non-null int64
keywords                4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
tagline                 3959 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
cast                    4803 non-null object
crew                    4803 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 562.9+ KB


In [20]:
df.drop(columns=['release_date'],inplace=True)

In [21]:
df['tagline'].fillna('',inplace=True)
df['overview'].fillna('',inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 13 columns):
genres                  4803 non-null object
movie_id                4803 non-null int64
keywords                4803 non-null object
original_title          4803 non-null object
overview                4803 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
tagline                 4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
cast                    4803 non-null object
crew                    4803 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 525.3+ KB


In [23]:
df.head(1)

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,tagline,vote_average,vote_count,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",Enter the World of Pandora.,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."


In [24]:
df['genres'] = df['genres'].str.join(' ').str.lower()

In [25]:
df['keywords'] = df['keywords'].str.join(' ').str.lower()

In [26]:
def remove_spaces(row):
    x=[]
    for i in row:
        i=i.replace(' ','')
        i=i.lower()
        x.append(i)
    return(x)

In [27]:
df['production_countries'] = df['production_countries'].apply(remove_spaces).str.join(' ')

In [28]:
df['production_companies'] = df['production_companies'].apply(remove_spaces).str.join(' ')

In [29]:
df['cast'] = df['cast'].apply(remove_spaces).str.join(' ')

In [30]:
df['crew'] = df['crew'].apply(remove_spaces).str.join(' ')

In [31]:
df['overview'] = df['overview'].str.lower()

In [32]:
df['tagline'] = df['tagline'].str.lower()

In [33]:
df['original_title'] = df['original_title'].str.lower()

In [34]:
df.head()

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,tagline,vote_average,vote_count,cast,crew
0,action adventure fantasy science fiction,19995,culture clash future space war space colony so...,avatar,"in the 22nd century, a paraplegic marine is di...",150.437577,ingeniousfilmpartners twentiethcenturyfoxfilmc...,unitedstatesofamerica unitedkingdom,enter the world of pandora.,7.2,11800,samworthington zoesaldana sigourneyweaver step...,stephene.rivkin rickcarter christopherboyes ch...
1,adventure fantasy action,285,ocean drug abuse exotic island east india trad...,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...",139.082615,waltdisneypictures jerrybruckheimerfilms secon...,unitedstatesofamerica,"at the end of the world, the adventure begins.",6.9,4500,johnnydepp orlandobloom keiraknightley stellan...,dariuszwolski goreverbinski jerrybruckheimer t...
2,action adventure crime,206647,spy based on novel secret agent sequel mi6 bri...,spectre,a cryptic message from bond’s past sends him o...,107.376788,columbiapictures danjaq b24,unitedkingdom unitedstatesofamerica,a plan no one escapes,6.3,4466,danielcraig christophwaltz léaseydoux ralphfie...,thomasnewman sammendes annapinnock johnlogan j...
3,action crime drama thriller,49026,dc comics crime fighter terrorist secret ident...,the dark knight rises,following the death of district attorney harve...,112.312950,legendarypictures warnerbros. dcentertainment ...,unitedstatesofamerica,the legend ends,7.6,9106,christianbale michaelcaine garyoldman annehath...,hanszimmer charlesroven christophernolan chris...
4,action adventure science fiction,49529,based on novel mars medallion space travel pri...,john carter,"john carter is a war-weary, former military ca...",43.926995,waltdisneypictures,unitedstatesofamerica,"lost in our world, found in another.",6.1,2124,taylorkitsch lynncollins samanthamorton willem...,andrewstanton andrewstanton johnlasseter colin...


In [35]:
r=Rake()

In [36]:
def extract_important_words(row):
    word = r.extract_keywords_from_text(row)
    word = r.get_ranked_phrases()
    return ' '.join(word)

In [37]:
df['overview'] = df['overview'].apply(extract_important_words)

In [38]:
df['tagline'] = df['tagline'].apply(extract_important_words)

In [39]:
df['keywords'] = df['keywords'].apply(extract_important_words)

In [40]:
df.head()

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,tagline,vote_average,vote_count,cast,crew
0,action adventure fantasy science fiction,19995,culture clash future space war space colony so...,avatar,unique mission paraplegic marine moon pandora ...,150.437577,ingeniousfilmpartners twentiethcenturyfoxfilmc...,unitedstatesofamerica unitedkingdom,world pandora enter,7.2,11800,samworthington zoesaldana sigourneyweaver step...,stephene.rivkin rickcarter christopherboyes ch...
1,adventure fantasy action,285,life traitor shipwreck strong woman ship allia...,pirates of the caribbean: at world's end,long believed elizabeth swann come back captai...,139.082615,waltdisneypictures jerrybruckheimerfilms secon...,unitedstatesofamerica,adventure begins world end,6.9,4500,johnnydepp orlandobloom keiraknightley stellan...,dariuszwolski goreverbinski jerrybruckheimer t...
2,action adventure crime,206647,novel secret agent sequel mi6 british secret s...,spectre,terrible truth behind spectre secret service a...,107.376788,columbiapictures danjaq b24,unitedkingdom unitedstatesofamerica,one escapes plan,6.3,4466,danielcraig christophwaltz léaseydoux ralphfie...,thomasnewman sammendes annapinnock johnlogan j...
3,action crime drama thriller,49026,dc comics crime fighter terrorist secret ident...,the dark knight rises,gotham city police department district attorne...,112.312950,legendarypictures warnerbros. dcentertainment ...,unitedstatesofamerica,legend ends,7.6,9106,christianbale michaelcaine garyoldman annehath...,hanszimmer charlesroven christophernolan chris...
4,action adventure science fiction,49529,novel mars medallion space travel princess ali...,john carter,reluctantly becomes embroiled former military ...,43.926995,waltdisneypictures,unitedstatesofamerica,world lost found another,6.1,2124,taylorkitsch lynncollins samanthamorton willem...,andrewstanton andrewstanton johnlasseter colin...


In [41]:
df['combined_features'] = df['genres']+' '+df['keywords']+' '+df['overview']+' '+df['tagline']+' '+df['cast']+' '+df['crew']

In [42]:
df.head()

,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,tagline,vote_average,vote_count,cast,crew,combined_features
0,action adventure fantasy science fiction,19995,culture clash future space war space colony so...,avatar,unique mission paraplegic marine moon pandora ...,150.437577,ingeniousfilmpartners twentiethcenturyfoxfilmc...,unitedstatesofamerica unitedkingdom,world pandora enter,7.2,11800,samworthington zoesaldana sigourneyweaver step...,stephene.rivkin rickcarter christopherboyes ch...,action adventure fantasy science fiction cultu...
1,adventure fantasy action,285,life traitor shipwreck strong woman ship allia...,pirates of the caribbean: at world's end,long believed elizabeth swann come back captai...,139.082615,waltdisneypictures jerrybruckheimerfilms secon...,unitedstatesofamerica,adventure begins world end,6.9,4500,johnnydepp orlandobloom keiraknightley stellan...,dariuszwolski goreverbinski jerrybruckheimer t...,adventure fantasy action life traitor shipwrec...
2,action adventure crime,206647,novel secret agent sequel mi6 british secret s...,spectre,terrible truth behind spectre secret service a...,107.376788,columbiapictures danjaq b24,unitedkingdom unitedstatesofamerica,one escapes plan,6.3,4466,danielcraig christophwaltz léaseydoux ralphfie...,thomasnewman sammendes annapinnock johnlogan j...,action adventure crime novel secret agent sequ...
3,action crime drama thriller,49026,dc comics crime fighter terrorist secret ident...,the dark knight rises,gotham city police department district attorne...,112.312950,legendarypictures warnerbros. dcentertainment ...,unitedstatesofamerica,legend ends,7.6,9106,christianbale michaelcaine garyoldman annehath...,hanszimmer charlesroven christophernolan chris...,action crime drama thriller dc comics crime fi...
4,action adventure science fiction,49529,novel mars medallion space travel princess ali...,john carter,reluctantly becomes embroiled former military ...,43.926995,waltdisneypictures,unitedstatesofamerica,world lost found another,6.1,2124,taylorkitsch lynncollins samanthamorton willem...,andrewstanton andrewstanton johnlasseter colin...,action adventure science fiction novel mars me...


In [43]:
df.drop(columns=['genres','keywords','overview','tagline','cast','crew'],inplace=True)

In [44]:
df.head(10)

,movie_id,original_title,popularity,production_companies,production_countries,vote_average,vote_count,combined_features
0,19995,avatar,150.437577,ingeniousfilmpartners twentiethcenturyfoxfilmc...,unitedstatesofamerica unitedkingdom,7.2,11800,action adventure fantasy science fiction cultu...
1,285,pirates of the caribbean: at world's end,139.082615,waltdisneypictures jerrybruckheimerfilms secon...,unitedstatesofamerica,6.9,4500,adventure fantasy action life traitor shipwrec...
2,206647,spectre,107.376788,columbiapictures danjaq b24,unitedkingdom unitedstatesofamerica,6.3,4466,action adventure crime novel secret agent sequ...
3,49026,the dark knight rises,112.312950,legendarypictures warnerbros. dcentertainment ...,unitedstatesofamerica,7.6,9106,action crime drama thriller dc comics crime fi...
4,49529,john carter,43.926995,waltdisneypictures,unitedstatesofamerica,6.1,2124,action adventure science fiction novel mars me...
5,559,spider-man 3,115.699814,columbiapictures lauraziskinproductions marvel...,unitedstatesofamerica,5.9,3576,fantasy action adventure friend egomania sand ...
6,38757,tangled,48.681969,waltdisneypictures waltdisneyanimationstudios,unitedstatesofamerica,7.4,3330,animation family hostage magic horse fairy tal...
7,99861,avengers: age of ultron,134.279229,marvelstudios primefocus revolutionsunstudios,unitedstatesofamerica,7.3,6767,action adventure science fiction comic book vi...
8,767,harry potter and the half-blood prince,98.885637,warnerbros. heydayfilms,unitedkingdom unitedstatesofamerica,7.4,5293,adventure fantasy family witchcraft wizardry a...
9,209112,batman v superman: dawn of justice,155.790452,dccomics atlasentertainment warnerbros. dcente...,unitedstatesofamerica,5.7,7004,action adventure fantasy comic book revenge su...


In [45]:
cv = CountVectorizer()

In [46]:
count_matrix = cv.fit_transform(df['combined_features'])

In [47]:
count_matrix.toarray().shape

(4803, 126971)

In [69]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='cosine',algorithm='brute')
model.fit(count_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [97]:
print('Enter Your Choice : ')
choice = input().lower()

Enter Your Choice : 
spider-man


In [98]:
choice_index = df[df['original_title'] == choice].index.values[0]

In [99]:
distances,indices = model.kneighbors(count_matrix[choice_index],n_neighbors=11)

In [100]:
count_matrix[choice_index].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [101]:
distances

array([[0.        , 0.78256936, 0.8053882 , 0.88313667, 0.89910374,
        0.90442045, 0.90521296, 0.90623505, 0.90804585, 0.91200286,
        0.91598027]])

In [102]:
indices

array([[159,  30,   5,  20, 215,  33,  38,  68, 174, 165, 511]])

In [103]:
x=0
for i in indices.flatten():
    if x==0:
        print('Recommendation for "{0}":'.format(choice.title()))
        x+=1
    else:
        print('{0}.  {1} '.format(x,df[df.index==i]['original_title'].values[0]).title())
        x+=1

Recommendation for "Spider-Man":
1.  Spider-Man 2 
2.  Spider-Man 3 
3.  The Amazing Spider-Man 
4.  4: Rise Of The Silver Surfer 
5.  X-Men: The Last Stand 
6.  The Amazing Spider-Man 2 
7.  Iron Man 
8.  The Incredible Hulk 
9.  Hulk 
10.  X-Men 
